# 网络搜索

> 没有一条帆船能吹动自己行驶，它需要外面的风。 -- 因可觅[《量子离歌》](https://read.douban.com/ebook/12945743/)

网络搜索本身是一个很重的服务，它包含三个核心任务：网页爬取、网页索引和网页检索。**网站索引** 几乎只能由大公司完成。即使是 [SearXNG](https://github.com/searxng/searxng) 这样开源检索方案，本质依然是聚合第三方搜索引擎（如必应、搜狗）的结果，没有独立索引能力。也就是说，即使自建网络搜索功能，也难以绕开大公司的服务。如此，不如一开始就老老实实使用大公司的 API 来获取搜索结果。

本节介绍三种网络搜索方式：

- 使用 [DashScope](https://dashscope.aliyun.com/)
- 使用 [Tavily](https://www.tavily.com/)
- 使用 [DDGS](https://github.com/deedy5/ddgs)

**DashScope** 是阿里的 MaaS 平台，由于我们已经有了阿里的 `API_KEY`，所以这是最轻松的方式。**Tavily** 是 [LangChain 推荐](https://docs.langchain.com/oss/python/integrations/retrievers/tavily) 的搜索服务，免费额度较多，但需要花点时间申请 `API_KEY`。**DDGS** 是一个免费的搜索引擎，不需要 `API_KEY`，但搜索结果的质量较差。

下面来看这三种搜索服务的使用方法和效果。

In [1]:
import os
from dotenv import load_dotenv
from langchain_openai import ChatOpenAI
from langchain.agents import create_agent
from langchain_core.tools import tool

# 加载模型配置
_ = load_dotenv()

# 加载模型
llm = ChatOpenAI(
    model="qwen-max",
    temperature=0.7,
    api_key=os.getenv("DASHSCOPE_API_KEY"),
    base_url=os.getenv("DASHSCOPE_BASE_URL"),
)

In [2]:
# !pip install dashscope langchain-community tavily-python ddgs

## 一、使用 DashScope

首先安装 dashscope：

```bash
pip install dashscope
```

将 `Generation.call` 函数的网络参数 `enable_search` 设为 `True` 就能接入互联网。下面把这个函数做成工具，供 Agent 调用。

In [3]:
from dashscope import Generation

@tool
def dashscope_search(query: str) -> str:
    """
    使用夸克搜索 API 搜索互联网信息。
    """
    response = Generation.call(
        model='qwen-max',
        prompt=query,
        enable_search=True,
        result_format='message'
    )

    if response.status_code == 200:
        return response.output.choices[0].message.content
    else:
        return (
            "Search failed with status code: "
            f"{response.status_code}, message: {response.message}"
        )

# 创建 Agent
agent = create_agent(
    model=llm,
    tools=[dashscope_search],
    system_prompt="你是一个智能助手，回答前必须使用工具搜索互联网信息",
)

# 运行 Agent
response = agent.invoke(
    {"messages": [{
        "role": "user",
        "content": "告诉我今天的日期，以及今天最重要的一条新闻"
    }]}
)

In [4]:
# 获取 Agent 的全部回复
for message in response['messages']:
    message.pretty_print()

================================ Human Message =================================

告诉我今天的日期，以及今天最重要的一条新闻
================================== Ai Message ==================================
Tool Calls:
  dashscope_search (call_beaf2e33383e42858320de)
 Call ID: call_beaf2e33383e42858320de
  Args:
    query: 今天的日期
  dashscope_search (call_6a613d93c4fd4f33a0cf35)
 Call ID: call_6a613d93c4fd4f33a0cf35
  Args:
    query: 今天 最重要的新闻
================================= Tool Message =================================
Name: dashscope_search

今天的日期是2025年12月20日，星期六。这一天也是农历二零二五年十一月初一。
================================= Tool Message =================================
Name: dashscope_search

今天（2025年12月20日）有一些重要的新闻和事件，以下是其中一些较为突出的：

- **国内养老金标准提高**：中央财经委员会办公室明确将继续提高城乡居民的基础养老金标准，目前最低为每人每月143元。这反映了国家对提高居民生活水平的持续努力。

- **生育保险覆盖范围扩大**：全国医保工作会议提出了新的要求，计划在2026年将灵活就业人员、农民工以及新就业形态人员纳入生育保险覆盖范围内，这意味着更多人将能够享受到生育保险带来的福利。

- **食品安全标签更新**：国家卫健委宣布将在预包装食品营养标签中加入饱和脂肪及糖含量的信息，旨在帮助消费者更好地了解食品成分，促进健康饮食习惯。

- **技术创新与产业发展**：中国科学家首次实现

## 二、使用 Tavily

首先安装 tavily：

```bash
pip install langchain-community tavily-python
```

然后从 [Tavily](https://www.tavily.com/) 官网申请 `API_KEY`，填入下面的方框中。

In [5]:
import getpass
os.environ["TAVILY_API_KEY"] = getpass.getpass()

 ········


In [6]:
from langchain_community.retrievers import TavilySearchAPIRetriever

retriever = TavilySearchAPIRetriever(k=3)
query = "告诉我今天的日期，以及今天最重要的一条新闻"

返回的内容有点多，可以解开下面这行命令的注释查看👇

In [7]:
# retriever.invoke(query)

同样把 Tavily 作为工具加入 Agent 看看效果怎么样！

In [8]:
prompt_template = """基于下面的搜索结果回答问题：

搜索结果：{context}

问题：{question}"""

def format_docs(docs):
    """合并 Tavily 的搜索结果"""
    return "\n\n".join(doc.page_content for doc in docs)

@tool
def tavily_search(query: str) -> str:
    """
    使用 Tavily 搜索 API 搜索互联网信息。
    """
    retriever = TavilySearchAPIRetriever(k=3)
    contents = retriever.invoke(query)
    if len(contents) > 0:
        return format_docs(contents)
    else:
        return "没有搜索结果"

# 创建 Agent
agent = create_agent(
    model=llm,
    tools=[tavily_search],
    system_prompt="你是一个智能助手，回答前必须使用工具搜索互联网信息",
)

# 运行 Agent
response = agent.invoke(
    {"messages": [{
        "role": "user",
        "content": "告诉我今天的日期，以及今天最重要的一条新闻"
    }]}
)

In [9]:
# 获取 Agent 的全部回复
for message in response['messages']:
    message.pretty_print()

================================ Human Message =================================

告诉我今天的日期，以及今天最重要的一条新闻
================================== Ai Message ==================================
Tool Calls:
  tavily_search (call_350268758bfd40e89ad3a1)
 Call ID: call_350268758bfd40e89ad3a1
  Args:
    query: 今天的日期
  tavily_search (call_a8ec41f10e2f4ed6a7f114)
 Call ID: call_a8ec41f10e2f4ed6a7f114
  Args:
    query: 今天最重要的新闻
================================= Tool Message =================================
Name: tavily_search

今日及下一个节日 · 元旦 · 春节 · 劳动节 · 端午节. 2026年6月19日周五. 182天后. 当前日期概览. 与今天日期相关的关键

今天的日期，写. 不同的方式来显示显示当前的日期不同国家存在：这两个最有名的是小端（DD-MM-YYYY）和大端（YYYY-MM-DD）。 在8601 ISO格式，需要使用烤箱位数字的年份，并设置每

| 今天阳历 | 2025年12月15日 星期一 | 生肖属相 | 蛇 | 最新资讯 | 2018年巴西节日假期安排 |. | 今天阴历 | 2025年十月廿六(闰6月) | 星座 | 射手座 | 黄历中"立券"是什么意思 |. | 天干地支 | 乙巳年 戊子月 戊午日 | 节气 |  | 黄历中"开市"是什么意思 |. # 节日大全. Copygight © 2010-2025 http://www.jintianjihao.com 沪ICP备2021007858号-4 联系QQ：6522-5692. 万年历查询 历史上的今天 万年历 二维码 Win10之家 微信二维码生成器 健康饮食 美容护肤 算命 

## 三、使用 DDGS

首先安装 DDGS：

```bash
pip install ddgs
```

我们直接使用 DDGS 获取的摘要信息，没有做进一步网页爬取。下面将 DDGS 做成工具。

In [10]:
from ddgs import DDGS

# 创建 ddgs 客户端
ddgs = DDGS()

@tool
def ddgs_search(query: str) -> str:
    """
    使用 DDGS 搜索 API 搜索互联网信息。

    Args:
        query: 搜索关键词或问题。
        max_results: 返回的最大结果条数。

    Returns:
        包含每条搜索结果的标题、摘要与链接的字符串。
    """
    results = list(
        ddgs.text(
            query=query,
            region="wt-wt",  # wt-wt zh
            timelimit='y',
            safesearch='off',  # moderate off
            page=1,
            backend='auto',
            max_results=3,
        )
    )

    content = ""
    for i, r in enumerate(results, 1):
        content += f"【结果 {i}】\n"
        content += f"标题: {r['title']}\n"
        content += f"摘要: {r['body']}\n"
        content += f"链接: {r['href']}\n\n"

    return content

# 创建 Agent
agent = create_agent(
    model=llm,
    tools=[ddgs_search],
    system_prompt="你是一个智能助手，回答前必须使用工具搜索互联网信息",
)

# 运行 Agent
response = agent.invoke(
    {"messages": [{
        "role": "user",
        "content": "告诉我今天的日期，以及今天最重要的一条新闻"
    }]}
)

In [11]:
# 获取 Agent 的全部回复
for message in response['messages']:
    message.pretty_print()

================================ Human Message =================================

告诉我今天的日期，以及今天最重要的一条新闻
================================== Ai Message ==================================
Tool Calls:
  ddgs_search (call_1d79b80b81b84b89adeb09)
 Call ID: call_1d79b80b81b84b89adeb09
  Args:
    query: 今天的日期
    max_results: 1
  ddgs_search (call_96fbdb961a55410c99df4b)
 Call ID: call_96fbdb961a55410c99df4b
  Args:
    query: 今天最重要的新闻
    max_results: 1
================================= Tool Message =================================
Name: ddgs_search

【结果 1】
标题: excel怎么自动计算某日期到今天的时间间隔-百度经验
摘要: May 18, 2019 · 首先要了解TODAY（）函数。在excel或wps中，TODAY（）可以返回今天的日期值。这里举例，可以看到A1中自动返回了2019/5/18
链接: https://jingyan.baidu.com/article/d45ad14869204069552b802d.html

【结果 2】
标题: Excel表格如何插入日期和时间并实现自动更新 - 百度经验
摘要: Jan 20, 2021 · 在数据录入过程中，有时需要录入当前日期、时间，并实现自动更新，使用NOW函数和TODAY函数可以轻松实现。 NOW函数可以实现返回当前日期和时间所对应的序列号。 …
链接: https://jingyan.baidu.com/article/bad08e1e86348348c85121f2.html

【结果 3】
标题: Excel中TODAY函数显示当天的日期 - 百度